# Diffusion Equation (2D)

## Introduction

Assuming that the thermal properties are constant and that we only consider a radiogenic heat source, the 2-D heat conduction equation is given by:

$\begin{equation}
\frac{\partial{T}}{\partial{t}} = \kappa \left( \frac{\partial^2{T}}{\partial{x^2}} + \frac{\partial^2{T}}{\partial{y}^2}\right) + \frac{Q_{I^\textrm{C}}}{\rho_0 c_p},
\end{equation}$

where

* $\rho_0$ is the reference density [ kg/m<sup>3</sup> ],
* $c_p$ is the specific heat capacity [ J/kg/K ],
* $\kappa = k/\rho_0/c_p$ is the thermal diffusivity [ m<sup>2</sup>/s ], and
* $Q$ is the radiogenic heat production per unit volume [ W/m<sup>3</sup> ].

The equation describes the change in temperature with time under the assumption that temperature spreads solely by diffusion.

## The Problem

Assume that Equation $(1)$ is defined in a 2D rectangular domain with a side length $\left(L\right)$ of 200 km and a depth $\left(H\right)$ of 100 km (Fig. 1). The temperature at the surface $\left(T_{\textrm{Surf}}\right)$ is 0 °C and increases linearly within the domain with a thermal gradient. For the temperature boundary conditions on the sides, we assume *Neumann* conditions. At a depth of $z_a = H/2$, we assume a horizontally oriented anomaly with a thickness $\left(h_a\right)$ of 10 km, extending across the entire width of the domain (e.g., a sill). The anomaly has an initial temperature $\left(T_a\right)$ of 1600 °C and a volumetric heat production rate $\left(Q_a\right)$ of $4.7 \cdot 10^{-6}$.

<img src="../Figures/Exercise05b_1.png" alt="drawing" width="450"/> <br>
**Fig. 1.** Model setup

Over time, the sill cools and thereby heats the surrounding rock. Due to the additional heat production rate of the sill, the temperature later increases again until a thermal equilibrium is established.


## The Solution

### Discretization

To solve the problem numerically, we need to subdivide our model domain into a numerical grid. We assume that the temperature and all other thermal properties are defined at the so-called *cell-centered* grid points (*centroids*) (see Fig. 2). This subdivision corresponds to that used for the [time-independent problem](./04_2D_Diffusion_Stationary_en.ipynb).

To solve our problem, we again make use of the *ghost nodes* of our grid, which allow a correct incorporation of the boundary conditions.

#### Grid and Indexing

<img src="../Figures/Exercise04_2.png" alt="drawing" width="450"/> <br>
**Fig. 2.** Staggered grid. The temperature is defined at the cell-centered grid points, the centroids (red circles). The gray circles outside the model domain are the ghost nodes.

The given staggered grid enables a so-called *conservative* finite difference approximation, where it is assumed that the heat flux $q_{i,j} = -k \frac{\partial{T}}{\partial{x_{i,j}}}$ is defined at the midpoint of the grid lines and the temperature is defined at the center of a grid cell (strictly speaking, the thermal conductivity is then also defined on the grid lines; however, since it is constant, we do not need to consider this here; for variable thermal parameters, the equation must be discretized somewhat differently; see the [documentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/DiffTwoD)). Moreover, using cell-centered temperature grid points in combination with *ghost nodes* allows us to incorporate boundary conditions relatively easily, while retaining the same order of accuracy as the central difference quotients in the interior of our model.

When indexing our grid points, we distinguish between *local* and *global* indices. The local index describes the position on the $i$,$j$ grid. The global index is a running index from 1 to $nc_x \cdot nc_y$, where $nc_i$ is the number of centroids in the respective spatial direction, and corresponds to the number of equations, i.e., the total number of interior grid points. The global index is also used when assembling the coefficient matrix for our linear system of equations.

For each central grid point, i.e., for each equation, a so-called numerical stencil specifies the positions of the grid points that are relevant for the respective equation. The coefficients for these grid points are then non-zero, while all others are zero. The nomenclature for the stencil often follows that of a compass, i.e., we have points to the south, west, center, east, and north (see Fig. 2). For each equation, the global index ($I$) of each stencil point is given by its relative position with respect to the central point ($I^\textrm{C}$) of the stencil, i.e.:

$\begin{equation}\begin{split}
I^\textrm{S} & = I^\textrm{C} - nc_x,\\
I^\textrm{W} & = I^\textrm{C} - 1,\\
I^\textrm{C} & = I, \\
I^\textrm{E} & = I^\textrm{C} + 1,\\
I^\textrm{N} & = I^\textrm{C} + nc_x,
\end{split}\end{equation}$

where $nc_x$ is the number of horizontal centroids, $I^\textrm{C}$ is the central reference point, and $I^\textrm{S}, I^\textrm{W}, I^\textrm{E}, I^\textrm{N}$ are the points to the south, west, east, and north of it.


#### Finite Difference Approximation

We can now approximate the partial differential equation using our finite differences.

*Explicit (Forward Euler scheme)*
$\begin{equation}
T_{I^\textrm{C}}^{n+1} = T_{I^\textrm{C}}^n + \kappa \Delta{t} \left(\frac{T_{I^\textrm{W}}^{n} - 2 T_{I^\textrm{C}}^{n} + T_{I^\textrm{E}}^{n}}{\Delta{x^2}} \right) + \kappa \Delta{t} \left(\frac{T_{I^\textrm{S}}^{n} - 2 T_{I^\textrm{C}}^{n} + T_{I^\textrm{N}}^{n}}{\Delta{y^2}} \right) + \frac{Q_{I^\textrm{C}} \Delta{t}}{\rho_0 c_p},
\end{equation}$

where $n+1$ and $n$ denote the new and the current time step, respectively.

*Implicit (Backward Euler scheme)*
$\begin{equation}
-b T_{I^\textrm{S}}^{n+1} - a T_{I^\textrm{W}}^{n+1} + (2a + 2b + c) T_{I^\textrm{C}}^{n} - a T_{I^\textrm{E}}^{n+1} - b T_{I^\textrm{N}}^{n+1} = cT_{I^\textrm{C}}^{n} + \frac{Q_{I^\textrm{C}}}{\rho_0 c_p},
\end{equation}$

where

$\begin{equation}\begin{split}
a & = \kappa / \Delta{x^2}, \\
b & = \kappa / \Delta{y^2}, \textrm{ and } \\
c & = 1 / \Delta{t}.
\end{split}\end{equation}$

These equations form a five-diagonal system of equations of the form:

$\begin{equation}
\mathbf{K} \cdot \bm{x} = \bm{b}
\end{equation}$

where $\mathbf{K}$ is the coefficient matrix (with five non-zero diagonals), $\bm{x}$ is the unknown solution vector (the temperatures at the cell centers at time step $n+1$), and $\bm{b}$ is the known right-hand side (the current temperature at the centroids).

**General Solution**

*Defect correction*

As in the 1D problem, the system of equations can be solved in a general way using defect correction. The heat conduction equation is reformulated by introducing a residual term $r$, which quantifies the deviation from the exact solution and can be iteratively reduced through successive correction steps to improve accuracy. In implicit form, the residual can be computed as:

$\begin{equation}
\mathbf{K} \cdot \bm{x} - \bm{b} = \bm{r},
\end{equation}$

where $\bm{r}$ is the residual.

The coefficients of the matrix are the same as derived above (Equation (5)). Within the defect correction framework, the coefficient matrix for a 2D problem with constant thermal properties can be assembled using the built-in function `AssembleMatrix2Dc()`.

After some algebra, Equation (7) yields the correction term for the initial temperature guess:

$\begin{equation}
\delta \bm{T} = -\mathbf{K}^{-1} \bm{r}^k
\end{equation}$

and the updated temperature after one iteration step:

$\begin{equation}
\bm{T}^{k+1} = \bm{T}^k + \delta \bm{T}.
\end{equation}$

---

Within `GeoModBox.jl`, the residual $\bm{r}$ is computed at the centroids using the extended temperature field of the current time step, which includes the ghost node temperature values, as an initial temperature guess:

$\begin{equation}
\frac{\partial{T_{\textrm{ext},I}}}{\partial{t}} - \kappa \left( \frac{\partial^2{T_{\textrm{ext},I}}}{\partial{x}^2} + \frac{\partial^2{T_{\textrm{ext},I}}}{\partial{y}^2} \right) - \frac{Q_{I^\textrm{C}}}{\rho_0 c_p} = \bm{r}_{I},
\end{equation}$

where $I$ is the equation number. Discretizing the equation in space and time using implicit finite differences yields:

$\begin{equation}
\frac{T_{\textrm{ext},I^\textrm{C}}^{n+1}-T_{\textrm{ext},I^\textrm{C}}^{n}}{\Delta{t}} - \kappa
\left( \frac{T_{\textrm{ext},I^\textrm{W}}^{n+1} - 2 T_{\textrm{ext},I^\textrm{C}}^{n+1} + T_{\textrm{ext},I^\textrm{E}}^{n+1}}{\Delta{x}^2} + \frac{T_{\textrm{ext},I^\textrm{S}}^{n+1} - 2 T_{\textrm{ext},I^\textrm{C}}^{n+1} + T_{\textrm{ext},I^\textrm{N}}^{n+1}}{\Delta{y}^2}
\right) - \frac{Q_{I^\textrm{C}}}{\rho_0 c_p} = \bm{r}_{I},
\end{equation}$

where $I^\textrm{C}$ is the global central reference point of the five-point stencil on the extended temperature field for the centroids. Rearranging Equation (11) and substituting the coefficients using Equation (5) gives:

$\begin{equation}
-b T_{\textrm{ext},I^\textrm{S}}^{n+1} - a T_{\textrm{ext},I^\textrm{W}}^{n+1} +
\left(2a + 2b + c \right) T_{\textrm{ext},I^\textrm{C}}^{n+1} -
a T_{\textrm{ext},I^\textrm{E}}^{n+1} - b T_{\textrm{ext},I^\textrm{N}}^{n+1} -
c T_{\textrm{ext},I^\textrm{C}}^n - \frac{Q_{I^\textrm{C}}}{\rho_0 c_p} =
r_{I},
\end{equation}$

which corresponds to the matrix form of Equation (7), where $T_{\textrm{ext},I}^{n+1}$ is the unknown vector $\bm{x}$ (using only the centroids), $-cT_{\textrm{ext},I}^n -\frac{Q_{I^\textrm{C}}}{\rho_0 c_p}$ is the known vector $\bm{b}$, and $-a$, $-b$, and $\left(2a+2b+c\right)$ are the coefficients of the non-zero diagonals of the coefficient matrix $\bm{K}$.

The residual can be computed using the built-in function `ComputeResiduals2Dc!()`. With the residual vector $\bm{r}$ and the coefficient matrix $\bm{K}$, the correction term for the temperature can be computed via Equation (8). The correction is then used to update the initial temperature guess (Equation (9)). This is repeated until the residual is considered sufficiently small.

**Special Solution**

If the problem is linear and the exact solution is reached within a single iteration step, the system of equations reduces to Equation (6). Thus, the system can be solved directly via a *left matrix division*:

$\begin{equation}
\bf{x} = \mathbf{K}^{-1} \bm{b}.
\end{equation}$

The coefficient matrix remains unchanged, even for the given boundary conditions. However, the right-hand side must be updated accordingly (setting $\bm{r}=0$ and adding the known parameters to the right-hand side of the equations). The special solution using left matrix division is implemented in the built-in functions `ForwardEuler2Dc!()` and `BackwardEuler2Dc!()`.

In this exercise, both solution approaches can be used, where the combined solver is included in the general solution. For more information on solving PDEs using different finite difference discretizations, including the corresponding conditions as well as advantages and disadvantages, please refer to the relevant [documentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/DiffTwoD).


To solve the problem using Julia, we first need to define the required modules (`ExtendableSparse, Plots`) and submodules (`GeoModBox.HeatEquation.TwoD`):

In [ ]:
using Plots, GeoModBox.HeatEquation.TwoD, ExtendableSparse
using Printf, LinearAlgebra
start = time()

We now define the model ($L$, $H$) and physical parameters ($k$, $Q$), as well as the discretization approach. We can choose the discretization method by defining the variable `FDSchema` as follows:

* A specialized solver for a linear problem (`explicit` or `implicit`)
* A combined, general solver with *defect correction* (`dc`)

In [ ]:
FDSchema    = :dc
# Physical parameters --------------------------------------------------- #
P      = (
    L       =   200e3,          # Length of the model [m]
    H       =   100e3,          # Height of the model [m]
    k       =   6,              # Thermal conductivity [W·m⁻¹·K⁻¹]
    cp      =   1000,           # Heat capacity [J·kg⁻¹·K⁻¹]
    ρ       =   3200,           # Density [kg·m⁻³]
    K0      =   273.15,         # Kelvin offset at 0 °C
    Q0      =   0,              # Background heat production rate [W·m⁻³]
)
P1      = (
    κ       =   P.k / P.ρ / P.cp,   # Thermal diffusivity [m²·s⁻¹]
    Tbot    =   1300.0 + P.K0,      # Temperature at the bottom boundary [K]
    Ttop    =   0.0 + P.K0,         # Temperature at the top boundary [K]
    Tplume  =   2000.0 + P.K0,      # Temperature of the plume [K]
    Xanoma  =   P.L/2.0,            # x-coordinate of the anomaly center [m]
    Wanoma  =   P.L,                # Width of the anomaly [m]
    Hanoma  =   10e3,               # Height (thickness) of the anomaly [m]
    Danoma  =   P.H/2.0,            # Depth of the anomaly center [m]
    Tanoma  =   1600.0 + P.K0,      # Temperature of the anomaly [K]
    Qanoma  =   2.7e-6              # Volumetric heat production rate [W·m⁻³]
)
P   = merge(P,P1)
# ----------------------------------------------------------------------- #


Next, we define the number of grid points and the grid spacing:

In [ ]:
# Numerical parameters -------------------------------------------------- #
NC      = (
    x       =   100,             # Grid points in the x-direction
    y       =   50,              # Grid points in the y-direction
)
Δ       = (
    x       = P.L/NC.x,          # Grid spacing in the x-direction [m]
    y       = P.H/NC.y           # Grid spacing in the y-direction [m]
)
# ----------------------------------------------------------------------- #


With these, the numerical grid can be defined (see Fig. 2):

In [ ]:
# Grid generation ------------------------------------------------------- #
x       = (
    c       =   LinRange(0.0 + Δ.x/2.0, P.L - Δ.x/2.0, NC.x),
)
y       = (
    c       =   LinRange(-P.H + Δ.y/2.0, 0.0 - Δ.y/2.0, NC.y),
)
# ----------------------------------------------------------------------- #

Since we are considering a time-dependent problem, we next need to define the time parameters. Note that for the explicit finite difference scheme, we must take the diffusion stability criterion into account. For more information on the conditions as well as the advantages and disadvantages of the discretization schemes for the 2D heat equation, please refer to the corresponding [documentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/DiffTwoD).

In [ ]:
# Time parameters ------------------------------------------------------- #
T       = (
    dn      =   25,             # Increment for graphical output,
                                # i.e., here only every 25th time step
    year    =   365.25*3600*24, # Seconds per year    
    dtfac   =   0.9,            # Multiplication factor for Δt
    Δ       =   [0.0],          # Time step length [s]
    nt      =   [0]             # Number of time steps (initialized)
)
T1      = (
    tmax    =   60 * 1e6 * T.year,    # Maximum simulation time [s]
    Δ       =   T.dtfac * (1.0 / (2.0 * P.κ * (1.0/Δ.x^2 + 1.0/Δ.y^2)))
)
T           = merge(T,T1)

nt          = floor(Int, T.tmax/T.Δ)  # Total number of time steps
Time        = zeros(nt)
# ----------------------------------------------------------------------- #

To visualize the results as an animation in a GIF file, the file location and name must be specified. As usual, the parameter `save_fig` determines whether the result should be saved as a *GIF* (1) or not (0).

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("05_2D_Sill_",FDSchema)
save_fig    =   1
# ----------------------------------------------------------------------- #

Next, we define the initial conditions and the data arrays for our problem:

In [ ]:
# Initial temperature field --------------------------------------------- #
D       =   (
    Q           =   zeros(NC...),
    T           =   zeros(NC...),
    T0          =   zeros(NC...),
    T_ex        =   zeros(NC.x+2,NC.y+2),
    T_ex0       =   zeros(NC.x+2,NC.y+2),
    Tmax        =   zeros(nt),
    Tprofile    =   zeros(NC.y, nt),
)
# Background field for heat sources
D.Q     .=  P.Q0
        
# Lithosphere temperature – linearly increasing with depth
for i = 1:NC.x, j = 1:NC.y
    D.T[i,j]     =   P.Ttop + abs(y.c[j]/P.H) * P.Tbot
    if abs(y.c[j] + P.Danoma) <= P.Hanoma/2.0
        D.T[i,j]    =   P.Tanoma
        D.Q[i,j]    =   P.Qanoma
    end    
end
@. D.T0                     =   D.T
D.T_ex[2:end-1,2:end-1]     .=  D.T

# Visualize initial condition ------------------------------------------- #
p = heatmap(x.c ./ 1e3, y.c ./ 1e3, (D.T .- P.K0)', 
        color=:viridis, colorbar=true, aspect_ratio=:equal, 
        xlabel="x [km]", ylabel="z [km]", 
        title="Temperature", 
        xlims=(0, P.L/1e3), ylims=(-P.H/1e3, 0.0), 
        clims=(0, 2000))

contour!(p, x.c./1e3, y.c/1e3, D.T' .- P.K0, levels=:10, linecolor=:black)
if save_fig == 0
    display(p)
end
# ----------------------------------------------------------------------- #

### Boundary Conditions

The temperature at the *ghost nodes* is defined for *Dirichlet* and *Neumann* boundary conditions in the same way as in the [steady-state 2-D case](./04_2D_Diffusion_Stationary_en.ipynb), and in the corresponding 1-D [explicit](./02_1D_Heat_explicit_en.ipynb) and [implicit](./03_1D_Heat_implicit_en.ipynb) versions. When initializing the boundary conditions, we make use of the tuple `BC`. The tuple defines the type of boundary condition (`type`) and the required value (`val`).

> Note: The solvers are designed such that boundary condition values are defined at every centroid; that is, the boundary condition must be provided as a 1D array.

In [ ]:
# Boundary Conditions --------------------------------------------------- #
BC      =   (type    = (W=:Neumann, E=:Neumann, N=:Dirichlet, S=:Dirichlet),
           val     = (W=zeros(NC.y),E=zeros(NC.y),N=D.T[:,end],S=D.T[:,1]))
# ----------------------------------------------------------------------- #

### System of Equations

For the implicit case, let us now initialize the coefficient matrix and the right-hand side:

In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   maximum(Num.T)
K       =   ExtendableSparseMatrix(ndof,ndof)
rhs     =   zeros(ndof)
# --- Defect Correction (dc) ---
niter   =   10
ϵ       =   1e-25
R       =   zeros(NC...)
∂2T     =   (∂x2=zeros(NC.x, NC.y), ∂y2=zeros(NC.x, NC.y),
                ∂x20=zeros(NC.x, NC.y), ∂y20=zeros(NC.x, NC.y))
# Define discretizationscheme ---
#   C   =   0   -> Implizit (default)
#   C   =   0.5 -> CNA
#   C   =   1.0 -> Explizit
C       =   0.0
# ----------------------------------------------------------------------- #

### Time Loop

Numerically, we can now solve the *PDE* in a time loop in different ways using the built-in functions from the `GeoModBox.HeatEquation.TwoD` submodule.

For the general solver using defect correction, we first compute the residual (`ComputeResiduals2Dc!()`), then assemble the coefficient matrix (`AssembleMatrix2Ds()`), compute the correction term $\delta T$, and finally update the temperature within an iteration loop (`1:niter`). This iteration is performed until the residual $R$ falls below a certain value $\epsilon$.

For the specialized solvers using left matrix division, we use the built-in functions for constant thermal properties, `ForwardEuler2Dc!()` and `BackwardEuler2Dc!()`.

In [ ]:
# Time loop ------------------------------------------------------------- #
for n = 1:nt
    println(n)
    # Store the temperature profile at x = L/2
    @. D.Tprofile[:, n] = (D.T[convert(Int, NC.x/2), :] +
                           D.T[convert(Int, NC.x/2) + 1, :]) / 2
    # Track the maximum of the mid-depth horizontal temperature
    D.Tmax[n] = maximum((D.T[:, convert(Int, NC.y/2)] +
                          D.T[:, convert(Int, NC.y/2) + 1]) / 2)

    if n > 1
        if FDSchema==:dc
            for iter = 1:niter
                # Evaluate residual
                ComputeResiduals2Dc!(R, D.T, D.T_ex, D.T0, D.T_ex0, ∂2T, 
                        P.κ, BC, Δ, T.Δ;C)
                @printf("||R|| = %1.4e\n", norm(R)/length(R))
                norm(R)/length(R) < ϵ ? break : nothing
                # Assemble linear system
                K  = AssembleMatrix2Dc(P.κ, BC, Num, NC, Δ, T.Δ;C)
                # Solve for temperature correction: Cholesky factorisation
                Kc = cholesky(K.cscmatrix)
                # Solve for temperature correction: Back substitutions
                δT = -(Kc\R[:])
                # Update temperature
                @. D.T += δT[Num.T]
            end
            D.T0    .= D.T
        elseif FDSchema==:explicit     
            ForwardEuler2Dc!( D, P.κ, Δ.x, Δ.y, T.Δ, NC, BC )        
        elseif FDSchema==:implicit
            BackwardEuler2Dc!( D, P.κ, Δ.x, Δ.y, T.Δ, NC, BC, rhs, K, Num )
        end             
        Time[n] = Time[n-1] + T.Δ
    end

    if mod(n, T.dn) == 0 || n == 1 || n == nt
        p = heatmap(x.c ./ 1e3, y.c ./ 1e3, (D.T .- P.K0)',
            color=:viridis, colorbar=true, aspect_ratio=:equal,
            xlabel="x [km]", ylabel="z [km]",
            title="Temperature",
            xlims=(0, P.L/1e3), ylims=(-P.H/1e3, 0.0),
            clims=(0, 1800))

        contour!(p, x.c ./ 1e3, y.c / 1e3, D.T' .- P.K0, levels=:10, linecolor=:black)
        if save_fig == 1
            Plots.frame(anim)
        else
            display(p)
        end
    end
end


### Results

For a clearer presentation and analysis of the results, we also plot a vertical temperature profile versus depth, as well as the maximum of a horizontal temperature profile versus time at the respective center of the domain:

In [ ]:
q = plot(D.Tprofile[:,1:T.dn:end].-P.K0,y.c./1e3,
        label="",xlabel="T_{x=L/2}",ylabel="Depth [km]",
        title="Temperature profile",
        layout=(1,2),subplot=1)

plot!(q,Time./T.year/1e6,D.Tmax.-P.K0,
        label="",xlabel="Time [My]",ylabel="T_{max} [°C]",
        subplot=2)

Now we still need to create and save the animation:

In [ ]:
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)        
    #savefig(string("./exercises/Correc Results/05_Sill_TProfile_Tmax_",FDSchema,".png"))
    savefig(string("./Results/05_Sill_TProfile_Tmax_",FDSchema,".png"))
else
    display(q)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
stop=time()
println(stop-start)